In [ ]:
import geopandas as gpd
import pandas as pd
import numpy as np
import torch
from torchvision import transforms
from torchvision.models import convnext_large, ConvNeXt_Large_Weights
from torch.utils.data import Dataset, DataLoader
from pathlib import Path
from PIL import Image
from tqdm import tqdm
from sklearn.decomposition import PCA

In [ ]:
class StreetViewDataset(Dataset):
    def __init__(self, dataframe, root_dir, transform=None):
        self.dataframe = dataframe
        self.root_dir = Path(root_dir)
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        row = self.dataframe.iloc[idx]
        images = []
        region_id = row['h3_9']
        for col in ['path_side_a', 'path_front', 'path_side_b', 'path_back']:
            img_path = self.root_dir / row[col]
            try:
                image = Image.open(img_path).convert('RGB')
                if self.transform:
                    image = self.transform(image)
                images.append(image)
            except FileNotFoundError:
                continue
        if not images:
            return None, None
        images_tensor = torch.stack(images)
        return images_tensor, region_id

In [ ]:
# Step 1: Prepare regions
regions_buffered_gdf = gpd.read_file("selected_regions_buffered_9.geojson")
regions_gdf = gpd.read_file("selected_regions_9.geojson")
regions_buffered_gdf.set_index('region_id', inplace=True)
regions_gdf.set_index('region_id', inplace=True)

# import panoids.geojson
path = "D://tu delft//Afstuderen//imagesummary//data//South_Holland_NL"
panoids_gdf = gpd.read_file(path + "//panoids//panoids.geojson")
image_root_dir = "D://tu delft//Afstuderen//imagesummary//data//South_Holland_NL//imagedb//"

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = convnext_large(weights=ConvNeXt_Large_Weights.DEFAULT)
model = torch.nn.Sequential(*(list(model.children())[:-1]))
model = model.to(device)
model.eval()

In [ ]:
# Transform setup
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Initialize the dataset and dataloader
dataset = StreetViewDataset(panoids_gdf, image_root_dir, transform=transform)
dataloader = DataLoader(dataset, batch_size=256, shuffle=False, pin_memory=True)

In [ ]:
def extract_embeddings(dataloader, model, device):
    mean_embeddings_list = []
    max_embeddings_list = []
    region_ids_list = []
    with torch.no_grad():
        for images, region_ids in tqdm(dataloader):
            if images is None:
                continue

            images = images.to(device, non_blocking=True)
            batch_mean_embeddings = []
            batch_max_embeddings = []
            for i in range(images.shape[0]):
                item_images = images[i]

                mean_aggregated_images = torch.mean(item_images, dim=0).unsqueeze(0)
                max_aggregated_images = torch.max(item_images, dim=0)[0].unsqueeze(0)

                mean_output = model(mean_aggregated_images)
                max_output = model(max_aggregated_images)

                batch_mean_embeddings.append(mean_output.cpu().numpy().flatten())
                batch_max_embeddings.append(max_output.cpu().numpy().flatten())

            mean_embeddings_list.extend(batch_mean_embeddings)
            max_embeddings_list.extend(batch_max_embeddings)
            if isinstance(region_ids, torch.Tensor):
                region_ids_list.extend(region_ids.cpu().numpy())
            else:
                region_ids_list.extend(region_ids)

    mean_embeddings_df = pd.DataFrame(mean_embeddings_list)
    max_embeddings_df = pd.DataFrame(max_embeddings_list)

    mean_embeddings_df['region_id'] = region_ids_list
    max_embeddings_df['region_id'] = region_ids_list

    mean_embeddings_df.set_index('region_id', inplace=True)
    max_embeddings_df.set_index('region_id', inplace=True)

    mean_embeddings_df.columns = [f'emb_mean_{i}' for i in range(mean_embeddings_df.shape[1])]
    max_embeddings_df.columns = [f'emb_max_{i}' for i in range(max_embeddings_df.shape[1])]

    return mean_embeddings_df, max_embeddings_df

In [ ]:
# Extract embeddings, applying max and mean aggregation to panoids
mean_embeddings_df, max_embeddings_df = extract_embeddings(dataloader, model, device)

# Check if the DataFrames are not None and print their shapes
if mean_embeddings_df is not None and max_embeddings_df is not None:
    print(f"Mean Embeddings DataFrame shape: {mean_embeddings_df.shape}")
    print(f"Max Embeddings DataFrame shape: {max_embeddings_df.shape}")
else:
    print("Error: Embeddings DataFrames are None")

In [ ]:
# Display the first few rows of the DataFrames
print("Mean Embeddings:")
print(mean_embeddings_df.head())
print("Max Embeddings:")
print(max_embeddings_df.head())

In [ ]:
# Save to CSV
mean_embeddings_df.to_csv('embeddings_streetview_mean_9.csv')
max_embeddings_df.to_csv('embeddings_streetview_max_9_convnext.csv')

In [ ]:
# PCA Plot (Optional)
from Plotting import pca_plot
import warnings
warnings.filterwarnings('ignore')
pca_plot(mean_embeddings_df, regions_buffered_gdf)

In [ ]:
pca_plot(max_embeddings_df, regions_buffered_gdf)